In [1]:
import requests
import time
import random

import pandas as pd

from bs4 import BeautifulSoup

In [3]:
#Scraper 1 (getting links to publications)

StartDate = '01/01/2000'
EndDate = '02/21/2023'

url = 'https://www.nist.gov/publications/search?k=&t=&a=&ps=All&n=&d%5Bmin%5D=' + StartDate + '&d%5Bmax%5D=' + EndDate

text = requests.get(url).text
soup = BeautifulSoup(text, 'html.parser')

nPages = int(str(soup.find_all('footer')[0])[36:-14]) // 25 + 1

d = []
t = []
l = []
for page in range(nPages):
    time.sleep(random.uniform(0.1, 2))
    url = 'https://www.nist.gov/publications/search?k=&t=&a=&ps=All&n=&d%5Bmin%5D=' + StartDate + '&d%5Bmax%5D=' + EndDate + '&page=' + str(page)
    text = requests.get(url).text
    soup = BeautifulSoup(text, 'html.parser')
    dates = soup.find_all('div', {"class": "nist-teaser__date"})
    for date in dates:
        s = str(date)[40:-13]
        d.append(s)
    titles = soup.find_all('h3', {"class": "nist-teaser__title"})
    for title in titles:
        s = str(title)[34:-9]
        s = s[s.find('>') + 1:]
        t.append(s)
        s = str(title)[40:] 
        s = 'https://www.nist.gov' + s[:s.find('>') - 1]
        l.append(s)

df = pd.DataFrame()
df['titles'] = t
df['links'] = l
df['dates'] = d
df.dates = pd.to_datetime(df.dates)   

In [5]:
#Scraper 2 (getting publication details)
df[['authors', 'abstracts', 'fields', 'keywords']] = ''

for i in range(len(df)):
    time.sleep(random.uniform(0.1, 0.3))
    url = df.links[i]
    text = requests.get(url).text
    soup = BeautifulSoup(text, 'html.parser')
    names = soup.find_all('div', {"class": "nist-field__item"})
    s = str(names[1]).replace('<span class="nist-author">', '').replace('</span>', '')[30:-6].replace(',   ', ', ').replace('  ', ' ')
    df.loc[i, 'authors'] = s
    abstract = soup.find_all('div', {"class": "text-with-summary"})
    df.loc[i, 'abstracts'] = str(abstract)[59:-10]
    if soup.find_all('div', {"class": "nist-tags"}):
        fields = soup.find_all('div', {"class": "nist-tags"})[0].find_all('a')
        f = ''
        for field in fields:
            f += field.text
            f += ', '
        df.loc[i, 'fields'] = f[:-2]
    keywords = soup.find_all('div', {"class": "nist-field__item"})
    df.loc[i, 'keywords'] = keywords[-1].text.replace(',  ', ', ')

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [6]:
df

,titles,links,dates,authors,abstracts,fields,keywords
0,NIST Metric Recipes,https://www.nist.gov/publications/nist-metric-...,2023-02-17,"Elizabeth Benham, Tanna Nguyen",Cooking and baking using a kitchen scale is a ...,"Weights and measures, Metric","metric system, SI, international, system, unit..."
1,Improved and Always Improving: Reference Formu...,https://www.nist.gov/publications/improved-and...,2023-02-16,"Allan H. Harvey, Jan Hruby, Karsten Meier",This article reviews the state of knowledge of...,"Thermodynamics, Reference data, Chemical therm...","D2O, H2O, heavy water, IAPWS, ice, molecular m..."
2,A Reference Equation of State with an Associat...,https://www.nist.gov/publications/reference-eq...,2023-02-15,"Kehui Gao, Jiangtao Wu, Ian Bell, Allan H. Har...",Fluid ammonia is highly associated because of ...,Thermodynamics,"Ammonia, Associating Fluids, Equation of State..."
3,Topological charge pumping with subwavelength ...,https://www.nist.gov/publications/topological-...,2023-02-15,"Ian Spielman, Gediminas Juzeliunas, Domantas B...",Recent experiments demonstrated deeply subwave...,"Quantum information science, Physics, Condense...","Quantum geometry, topology, Quantum Hall, Bose..."
4,Characterization of Reference Materials 8690 t...,https://www.nist.gov/publications/characteriza...,2023-02-13,"Jessica Reiner, Benjamin Place, N. Alan Hecker...",The National Institute of Standards and Techno...,"Reference materials, Environment, Analytical c...","aqueous film-forming foam (AFFF), high-resolut..."
...,...,...,...,...,...,...,...
41867,Waveguide atom beam splitter for laser-cooled ...,https://www.nist.gov/publications/waveguide-at...,2000-01-01,,,,
41868,Wear Analysis of UHMWPE Using a load Sum Method,https://www.nist.gov/publications/wear-analysi...,2000-01-01,,,,
41869,Well-ionization Chamber Response Relative to N...,https://www.nist.gov/publications/well-ionizat...,2000-01-01,,,,
41870,X-ray and extreme ultraviolet emission induced...,https://www.nist.gov/publications/x-ray-and-ex...,2000-01-01,,,,


In [7]:
df.to_csv('NIST_publications_2000-2023.csv')